In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torchvision import transforms
from tqdm import tqdm

In [21]:
PROJECT_ROOT = os.path.abspath("..")
PROCESSED_PATH = os.path.join(PROJECT_ROOT, "data", "processed")
TRAIN_CSV = os.path.join(PROCESSED_PATH, "train.csv")
VAL_CSV = os.path.join(PROCESSED_PATH, "val.csv")
TEST_CSV = os.path.join(PROCESSED_PATH, "test.csv")

In [22]:
train_df = pd.read_csv(TRAIN_CSV)
val_df = pd.read_csv(VAL_CSV)
test_df = pd.read_csv(TEST_CSV)

le = LabelEncoder()
le.fit(train_df["label"])
train_df["label_encoded"] = le.transform(train_df["label"])
val_df["label_encoded"] = le.transform(val_df["label"])
test_df["label_encoded"] = le.transform(test_df["label"])

In [23]:
IMAGE_SIZE = (48, 48)

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor()
])

In [24]:
def load_images(df, transform=None):
    images = []
    labels = []
    for row in tqdm(df.itertuples(), total=len(df)):
        try:
            img = Image.open(row.filepath)
            if transform:
                img = transform(img)
            images.append(img)
            labels.append(row.label_encoded)
        except:
            continue
    return torch.stack(images), torch.tensor(labels)

In [25]:
X_train, y_train = load_images(train_df, transform)

100%|██████████| 2474/2474 [00:01<00:00, 1573.99it/s]


In [26]:
X_val, y_val = load_images(val_df, transform)

100%|██████████| 530/530 [00:00<00:00, 1441.45it/s]


In [27]:
X_test, y_test = load_images(test_df, transform)

100%|██████████| 531/531 [00:00<00:00, 1407.29it/s]


In [28]:
X_train = X_train.float() / 255.0
X_val = X_val.float() / 255.0
X_test = X_test.float() / 255.0

In [29]:
summary = {
    "train_images": X_train.shape,
    "train_labels": y_train.shape,
    "val_images": X_val.shape,
    "val_labels": y_val.shape,
    "test_images": X_test.shape,
    "test_labels": y_test.shape,
    "num_classes": len(le.classes_)
}

print(summary)

{'train_images': torch.Size([2474, 1, 48, 48]), 'train_labels': torch.Size([2474]), 'val_images': torch.Size([530, 1, 48, 48]), 'val_labels': torch.Size([530]), 'test_images': torch.Size([531, 1, 48, 48]), 'test_labels': torch.Size([531]), 'num_classes': 2}
